In [1]:
import json
from textwrap import dedent

import duckdb
from phi.assistant import Assistant
from phi.assistant.duckdb import DuckDbAssistant
from phi.assistant.python import PythonAssistant
from phi.llm.ollama import Ollama

In [2]:
dedent(
    """
    # CONTEXT #
    Hello, World!

    # OBJECTIVE #
    This is a test
    """
)

'\n# CONTEXT #\nHello, World!\n\n# OBJECTIVE #\nThis is a test\n'

In [4]:
with duckdb.connect("broadcast.db") as con:
    con.sql(
        "create table if not exists broadcast as select * from '/Users/mraad/data/Broadcast.prq/part-*.parquet'"
    )

In [18]:
with duckdb.connect("broadcast.db", read_only=True) as con:
    print(
        con.sql(
            dedent(
                """
    SELECT EXTRACT(hour FROM BaseDateTime) AS HourOfDay, COUNT(*) AS RecordCount
    FROM Broadcast
    GROUP BY HourOfDay
    ORDER BY HourOfDay"""
            )
        )
    )

┌───────────┬─────────────┐
│ HourOfDay │ RecordCount │
│   int64   │    int64    │
├───────────┼─────────────┤
│         0 │       58105 │
│         1 │       56452 │
│         2 │       54895 │
│         3 │       54998 │
│         4 │       53428 │
│         5 │       52946 │
│         6 │       52486 │
│         7 │       53219 │
│         8 │       55741 │
│         9 │       58349 │
│         · │         ·   │
│         · │         ·   │
│         · │         ·   │
│        14 │       57395 │
│        15 │       56932 │
│        16 │       57902 │
│        17 │       57362 │
│        18 │       57872 │
│        19 │       58795 │
│        20 │       59099 │
│        21 │       58509 │
│        22 │       58778 │
│        23 │       59216 │
├───────────┴─────────────┤
│   24 rows (20 shown)    │
└─────────────────────────┘



In [9]:
# assistant = Assistant(
#     llm=Ollama(),
#     tools=[DuckDuckGo()],
#     show_tool_calls=True,
#     description="You are an expert on world affairs.",
# )

In [10]:
# assistant.print_response(
#     "Please tell me what is going on in Lebanon now.", markdown=True
# )

In [15]:
duckdb_assistant = DuckDbAssistant(
    # llm=Ollama(model="openhermes"),
    show_tool_calls=True,
    db_path="broadcast.db",
    read_only=True,
    create_tables=False,
    semantic_model=json.dumps(
        {
            "tables": [
                {
                    "name": "Broadcast",
                    "description": "Contains information about AIS broadcast points.",
                    "columns": [
                        {"name": "OBJECTID", "type": "INTEGER"},
                        {"name": "Longitude", "type": "DOUBLE"},
                        {"name": "Latitude", "type": "DOUBLE"},
                        {"name": "MMSI", "type": "INTEGER"},
                        {"name": "Status", "type": "INTEGER"},
                        {"name": "BaseDateTime", "type": "TIMESTAMP"},
                    ],
                }
            ]
        }
    ),
)

In [17]:
duckdb_assistant.print_response(
    "Count the number of broadcast records by each hour of the day"
)

Output()

INFO     Running: explain SELECT EXTRACT(hour FROM BaseDateTime) AS HourOfDay, COUNT(*) AS RecordCount FROM        
         Broadcast GROUP BY HourOfDay ORDER BY HourOfDay

INFO     Running: SELECT EXTRACT(hour FROM BaseDateTime) AS HourOfDay, COUNT(*) AS RecordCount FROM Broadcast GROUP
         BY HourOfDay ORDER BY HourOfDay